# Initilaisation

In [ ]:
import ROOT
# from T2KStyle import T2KStyle
c = ROOT.TCanvas('c', '', 800, 600)
cl = ROOT.TCanvas('cl', '', 1600, 1200)
cw = ROOT.TCanvas('c2', '', 1600, 600)

In [ ]:
# 0 - desy
# 1 - mm1
# 2 - er2
# 3 - er3

# 4 - DESY cosmic
# path = '~/LXPLUS/DESY_testbeam/cosmic_strict_v2/'
path = '/eos/user/s/ssuvorov/DESY_testbeam/cosmic_strict_v2/'
voltage = '380'

file = [ROOT.TFile(path + '../tree/g_' + voltage + '_200_dedx.root')]
file.append(ROOT.TFile(path + '/mm1_' + voltage + 'V_200ns_dedx.root'))
file.append(ROOT.TFile(path + '/er2_' + voltage + 'V_dedx.root'))
file.append(ROOT.TFile(path + '/er3_' + voltage + 'V_dedx.root'))

# file.append(ROOT.TFile('~/LXPLUS/DESY_testbeam/multi_pad/c_0d_0p2T_200ns_50Mhz_dedx.root'))
file.append(ROOT.TFile(path + '../tree/c_0d_0T_200ns_25Mhz_dedx.root'))

name = ['DESY beam', 'MM1', 'ERAM2', 'ERAM3', 'DESY_cosmic', 'DESY_cosmic 2']
color = [ROOT.kBlack, ROOT.kRed, ROOT.kGreen, ROOT.kBlue, ROOT.kViolet, ROOT.kCyan]

# Analysis

## Charge

In [ ]:
%jsroot on
cl.Clear()
cl.Divide(2, 2)

ROOT.gStyle.SetOptTitle(0)

hs = [ROOT.THStack("hs_charge", "")]
hs.append(ROOT.THStack("hs_dedx", ""));
hs.append(ROOT.THStack("hs_mult", ""))

x_axis = ['Untruncated charge [a.u.]', 'Truncated charge [a.u.]', 'Multiplicity']
x_axis_r = [[0., 5000.], [0., 2000.], [0., 10.]]

for i in [0, 1, 2, 3, 4]:
    charge = file[i].Get('un_trunc_cluster')
    charge.Scale(1./charge.Integral())
    charge.SetLineColor(color[i])
    charge.SetTitle(name[i])
    hs[0].Add(charge)


    dedx = file[i].Get('dEdx')
    dedx.Scale(1./dedx.Integral())
    dedx.SetLineColor(color[i])
    dedx.SetTitle(name[i])
    dedx.GetXaxis().SetRangeUser(0., 2000.)
    hs[1].Add(dedx)

    Mult = file[i].Get('Mult')
    Mult.Scale(1./Mult.Integral())
    Mult.SetLineColor(color[i])
    Mult.SetTitle(name[i])
    hs[2].Add(Mult)
    
for i, h in enumerate(hs):
    cl.cd(i+1)
    h.Draw('histo nostack')
    h.GetXaxis().SetTitle(x_axis[i])
    h.GetXaxis().SetRangeUser(x_axis_r[i][0], x_axis_r[i][1])
    ROOT.gPad.BuildLegend(0.5, 0.55, 0.8, 0.85)

    
cl.Draw()

In [ ]:
c.Clear()
c.cd()
hs = ROOT.THStack("hs_charge", "")
for i in [0, 1, 2, 3, 4]:
    dedx = file[i].Get('dEdx')
    dedx.Scale(1./dedx.Integral())
    dedx.SetLineColor(color[i])
    dedx.SetTitle(name[i])
    dedx.GetXaxis().SetRangeUser(0., 2000.)
    hs.Add(dedx)
    dedx.Fit('gaus')
    sigma = dedx.GetFunction('gaus').GetParameter(2)
    mean = dedx.GetFunction('gaus').GetParameter(1)
    print(100*sigma/mean)
hs.Draw('histo nostack')
hs.GetXaxis().SetRangeUser(0., 1600) 
c.Draw()

In [ ]:
%jsroot on
cw.Clear()
cw.cd()
cw.Divide(3)
ROOT.gStyle.SetOptTitle(1)

hs = []
for pad in range(3):
    cw.cd(pad+1)
    hs.append(ROOT.THStack('hs', ''))
    h = []
    for i in [0, 1, 2, 3]:
        if pad == 0:
            h.append(ROOT.TH1F('h', '', 100, 0., 2000.))
        else:
            h.append(ROOT.TH1F('h', '', 200, 0., 800.))
            
        file[i].outtree.Project('h', f'pad_charge[{pad}][]', f'multiplicity > {pad+1}')
        h[-1].Scale(1./h[-1].Integral())
        h[-1].SetTitle(name[i])
        h[-1].SetLineColor(color[i])
        hs[-1].Add(h[-1])
    
    hs[-1].Draw('histo nostack')
    hs[-1].GetXaxis().SetTitle('Charge [a.u.]')
    hs[-1].SetTitle(f'Pad {pad+1}')
    ROOT.gPad.BuildLegend()
cw.Draw()    

In [ ]:
h = ROOT.TH1F('h', '', 1, 0., 1.)
for i in [0, 1, 2, 3, 4]:
    pads = file[i].outtree.Project('h', 'pad_charge[3][]', 'multiplicity > 2')
    bad = file[i].outtree.Project('h', 'pad_charge[3][]', 'multiplicity > 2 && pad_time[2][] - pad_time[0][] < 2')
    print(bad / pads * 100)

## Angle

In [ ]:
cw.Clear()
cw.Divide(2)

cw.cd(1)
f_id = 1
angle = file[f_id].Get('angle')
angle.GetXaxis().SetRangeUser(0., 0.4)
angle.GetYaxis().SetRangeUser(0., 0.4)
angle.SetTitle(name[f_id])
angle.GetXaxis().SetTitle('tg(#phi)')
angle.GetYaxis().SetTitle('tg(#theta)')
angle.Draw('colz')

cw.cd(2)
f_id = 2
angle = file[f_id].Get('angle')
angle.GetXaxis().SetRangeUser(0., 0.4)
angle.GetYaxis().SetRangeUser(0., 0.4)
angle.SetTitle(name[f_id])
angle.GetXaxis().SetTitle('tg(#phi)')
angle.GetYaxis().SetTitle('tg(#theta)')
angle.Draw('colz')

cw.Draw()

## Gas quality

In [ ]:
%jsroot on
cl.Clear()
cl.cd()
cl.Divide(2, 2)

h2 = []

for i in [0, 1, 2, 3]:
    cl.cd(i+1)
    h2.append(ROOT.TH2F('h2', '', 511, 0., 511., 100, 0., 3000.))
    file[i].outtree.Project('h2', 'dEdx:pad_time[0][]', '', '')
    h2[i].GetXaxis().SetTitle('t_{1}')
    h2[i].GetYaxis().SetTitle('Cluster charge')
    h2[i].Draw('colz')
    
cl.Draw()

In [ ]:
%jsroot on
c.Clear()
c.cd()

h2 = []

hs = ROOT.THStack('hs', '')

for i in [1, 2, 3]:
#     cl.cd(i+1)
    h2.append(ROOT.TH1F('h2', '', 511, 0., 511.))
    file[i].outtree.Project('h2', 'pad_time[0][]', '', '')
    h2[-1].Scale(1./h2[-1].Integral())
    h2[-1].SetLineColor(color[i])
    h2[-1].SetTitle(name[i])
    hs.Add(h2[-1])

hs.Draw('histo nostack')
hs.GetXaxis().SetTitle('t_{1}')
c.BuildLegend()   
c.Draw()

In [ ]:
%jsroot on
cl.Clear()
cl.cd()
cl.Divide(2, 2)

for i in [0, 1, 2, 3]:
    cl.cd(i+1)
    h2 = ROOT.TH2F('h2', '', 511, 0., 511., 100, 0., 3000.)
    file[i].outtree.Project('h2', 'dEdx:pad_time[0][]', '', '')
    h2.GetXaxis().SetTitle('t_{1}')
    h2.GetYaxis().SetTitle('Cluster charge')
    h2.Draw('colz')
    
cl.Draw()